In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
!pip install transformers datasets

In [2]:
import nltk 
nltk.download()
import re

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


# Creating Dataset

In [47]:
import json
 
dataset = [
    {
        "scenario": "Product Sales",
        "dialogues": [
            {
                "dialogue": [
                    {"speaker": "Agent", "text": "Hello, this is Nandini from Amazon. How are you today?"},
                    {"speaker": "Customer", "text": "I'm good, thank you."},
                    {"speaker": "Agent", "text": "Great to hear! I'm calling to introduce you to our latest software solution that can help streamline your business operations. Are you currently using any software for project management?"},
                    {"speaker": "Customer", "text": "Yes, we are using Chrome."},
                    {"speaker": "Agent", "text": "That's great! Our software integrates seamlessly with Chrome and offers additional features such as advanced analytics and real-time collaboration. Would you be interested in a demo?"},
                    {"speaker": "Customer", "text": "Sure, I would like to see a demo."},
                    {"speaker": "Agent", "text": "Excellent! I can schedule a demo for you at your convenience. What time works best for you?"}
                ]
            },
            {
                "dialogue": [
                    {"speaker": "Agent", "text": "Hi, this is Nandini from Amazon. How are you today?"},
                    {"speaker": "Customer", "text": "I'm fine, who is this?"},
                    {"speaker": "Agent", "text": "I’m with Amazon, and we have a new project management tool that I think you'll find very useful. Are you currently managing projects manually or using software?"},
                    {"speaker": "Customer", "text": "We do it manually."},
                    {"speaker": "Agent", "text": "Our tool can automate a lot of the tasks you're doing manually, saving you time and reducing errors. Can I tell you more about how it works?"},
                    {"speaker": "Customer", "text": "Okay, I'm listening."},
                    {"speaker": "Agent", "text": "Great! Our software provides features like task automation, progress tracking, and team collaboration tools. Would you be interested in a free trial to see how it can benefit your work?"},
                    {"speaker": "Customer", "text": "Yes, a free trial sounds good."}
                ]
            }
        ]
    }]


In [6]:
dataset

[{'scenario': 'Product Sales',
  'dialogues': [{'dialogue': [{'speaker': 'Agent',
      'text': 'Hello, this is [Agent Name] from [Company Name]. How are you today?'},
     {'speaker': 'Customer', 'text': "I'm good, thank you."},
     {'speaker': 'Agent',
      'text': "Great to hear! I'm calling to introduce you to our latest software solution that can help streamline your business operations. Are you currently using any software for project management?"},
     {'speaker': 'Customer', 'text': 'Yes, we are using [Current Software].'},
     {'speaker': 'Agent',
      'text': "That's great! Our software integrates seamlessly with [Current Software] and offers additional features such as advanced analytics and real-time collaboration. Would you be interested in a demo?"},
     {'speaker': 'Customer', 'text': 'Sure, I would like to see a demo.'},
     {'speaker': 'Agent',
      'text': 'Excellent! I can schedule a demo for you at your convenience. What time works best for you?'}]},
   {'di

# Prepare the data

In [7]:
conversations = []
for scenario in dataset:
    for dialogue in scenario['dialogues']:
        for turn in dialogue['dialogue']:
            conversations.append(turn['text'])

# Train the Dialogue model

In [10]:
from datasets import Dataset

In [38]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, pipeline

# Tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-medium')

loading file vocab.json from cache at C:\Users\asus/.cache\huggingface\hub\models--microsoft--DialoGPT-medium\snapshots\7b40bb0f92c45fefa957d088000d8648e5c7fa33\vocab.json
loading file merges.txt from cache at C:\Users\asus/.cache\huggingface\hub\models--microsoft--DialoGPT-medium\snapshots\7b40bb0f92c45fefa957d088000d8648e5c7fa33\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\asus/.cache\huggingface\hub\models--microsoft--DialoGPT-medium\snapshots\7b40bb0f92c45fefa957d088000d8648e5c7fa33\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\asus/.cache\huggingface\hub\models--microsoft--DialoGPT-medium\snapshots\7b40bb0f92c45fefa957d088000d8648e5c7fa33\config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdr

In [39]:
# Add padding token 
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

Assigning [PAD] to the pad_token key of the tokenizer
Adding [PAD] to the vocabulary


Embedding(50258, 1024)

In [40]:
# Tokenize the dataset
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

tokenized_texts = tokenize_function(conversations)

# Convert to Dataset object
dataset = Dataset.from_dict(tokenized_texts)

In [12]:
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
tokens = tokenizer(conversations, return_tensors='pt', truncation=True, padding=True)

In [41]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False)

In [44]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [45]:
# Training model
trainer.train()

***** Running training *****
  Num examples = 15
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 354824192


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=24, training_loss=4.672073046366374, metrics={'train_runtime': 549.241, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.044, 'total_flos': 10447882813440.0, 'train_loss': 4.672073046366374, 'epoch': 3.0})

In [48]:
# Save the model and tokenize
model.save_pretrained("./cold_calling_model")
tokenizer.save_pretrained("./cold_calling_model")

Configuration saved in ./cold_calling_model\config.json
Model weights saved in ./cold_calling_model\pytorch_model.bin
tokenizer config file saved in ./cold_calling_model\tokenizer_config.json
Special tokens file saved in ./cold_calling_model\special_tokens_map.json
added tokens file saved in ./cold_calling_model\added_tokens.json


('./cold_calling_model\\tokenizer_config.json',
 './cold_calling_model\\special_tokens_map.json',
 './cold_calling_model\\vocab.json',
 './cold_calling_model\\merges.txt',
 './cold_calling_model\\added_tokens.json')

In [49]:
# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [50]:
# Generate a new dialogue
input_text = "Hello, this is Nandini from Amazon. How are you today?"
generated_text = text_generator(input_text, max_length=100, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [51]:
print(generated_text[0]['generated_text'])

Hello, this is Nandini from Amazon. How are you today?
